In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

### Gemini 3 and Gemini 2.5 Response Format 

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage

gemini3 = "gemini-3-pro-preview"
# gemini2 = "gemini-2.5-pro"
gemini2 = "gemini-2.5-flash"

system_msg = SystemMessage("You are a helpful assistant.")

query = "Explain the theory of relativity in simple terms."
messages = [system_msg, HumanMessage(query)]


In [ ]:
model = ChatGoogleGenerativeAI(model=gemini3)
response = model.invoke(messages)

In [ ]:
response

In [ ]:
response.text

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)
response = model.invoke(messages)

In [ ]:
response

In [ ]:
response.content
response.content_blocks
response.usage_metadata
response.response_metadata


### Streaming

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)

query = "Write a short story about the earth and the moon."
for chunk in model.stream(query):
    print(chunk.content, end="", flush=True)

### Multimodal usage

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini3)

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the image provided."},
        {
            "type": "image",
            "url": "https://www.shutterstock.com/image-vector/vector-cute-baby-panda-cartoon-600nw-2427356853.jpg",
        },
    ]
)
response = model.invoke([message])

In [ ]:
response.content
response.text

In [ ]:
## Reading media from local file and encoding to base64
## Now use smaller model for faster response

## image mime type example
# mime_type = "image/png"

## pdf mime type example
# mime_type = "application/pdf", type = "file"

## audio mime type example
# mime_type = "audio/mpeg", type = "audio"

import base64

image_bytes = open("data/images/panda.png", "rb").read()
bytes_base64 = base64.b64encode(image_bytes).decode("utf-8")

mime_type = "image/png"

message = HumanMessage(
    content=[
        {"type": "text", "text": "Describe the image provided."},
        {
            "type": "image",
            "base64": bytes_base64,
            "mime_type": mime_type,
        },
    ]
)

model = ChatGoogleGenerativeAI(model=gemini2)
response = model.invoke([message])

In [ ]:
response

In [ ]:
pdf_bytes = open("data/rag-data/apple/apple 10-q q1 2024.pdf", "rb").read()
pdf_base64 = base64.b64encode(pdf_bytes).decode("utf-8")

mime_type = "application/pdf"

message = HumanMessage(
    content=[
        {"type": "text", "text": "Summarize the key financial highlights from this quarterly report."},
        {
            "type": "file",
            "base64": pdf_base64,
            "mime_type": mime_type,
        },
    ]
)
response = model.invoke([message])

In [ ]:
response.text

In [ ]:
response.content
response.content_blocks
response.usage_metadata
response.response_metadata

### Tool Calling

In [ ]:
# first show ollama web_search tool
# then build weather tool

from scripts import base_tools

In [ ]:
response = base_tools.web_search.invoke({'query': 'what is the latest US stock market updates?'})
# response

In [ ]:
response = base_tools.get_weather.invoke({'location': 'Mumbai'})
# response

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)
model_with_tools = model.bind_tools([base_tools.web_search, base_tools.get_weather])

In [ ]:
model_with_tools
response = model_with_tools.invoke("what is the weather in Mumbai today? and latest news on US stock market?")

In [ ]:
response

### Thinking support

https://ai.google.dev/gemini-api/docs/thinking

With reasoning models, you have the option to adjust the number of internal thinking tokens used (thinking_budget) or to disable thinking altogether.


In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2,
                               thinking_budget=1024,
                               include_thoughts=True)

query = "explain the theory of relativity in simple terms."
response = model.invoke(query)

In [ ]:
print(response)
# print(response.text)

response.content_blocks

response

In [ ]:
model_with_tools = model.bind_tools([base_tools.web_search, base_tools.get_weather])

response = model_with_tools.invoke(query)


In [ ]:
response.content_blocks
response

### Built-in tools

Google Gemini supports a variety of built-in tools, which can be bound to the model in the usual way.

In [ ]:
model = ChatGoogleGenerativeAI(model=gemini2)

#  base_tools.get_weather -> function calling with tool use is unsupported.
model_with_tools = model.bind_tools([{"google_search": {}}, {"code_execution": {}}])

In [ ]:
query = "When is the next total solar eclipse in the US and what is 3 + 2?"
response = model_with_tools.invoke(query)

In [ ]:
print(response.text)

In [ ]:
response.response_metadata

### Context Caching

- https://ai.google.dev/gemini-api/docs/caching?hl=en&lang=python#pdfs_1
- https://ai.google.dev/gemini-api/docs/pricing

In [ ]:
import time
from google import genai
from google.genai.types import CreateCachedContentConfig, Content, Part

client = genai.Client()

In [ ]:
file_paths = [
    "data/rag-data/apple/apple 10-q q1 2024.pdf",
    "data/rag-data/apple/apple 10-q q2 2024.pdf"
]

uploaded_files = []
for path in file_paths:
    file = client.files.upload(file=path)
    while file.state.name == "PROCESSING":
        time.sleep(2)
        file = client.files.get(name=file.name)
    uploaded_files.append(file)

In [ ]:
parts = []
for f in uploaded_files:
    part = Part.from_uri(file_uri=f.uri, mime_type=f.mime_type)
    parts.append(part)

contents = [
    Content(
        role="user",
        parts=parts,
    )
]

cache = client.caches.create(
    model=gemini2,
    config=CreateCachedContentConfig(
        display_name="Apple Q1 Q2 2024 Reports",
        system_instruction="You are a financial analyst. Use these Apple quarterly reports to answer questions.",
        contents=contents,
        ttl="1800s",
    ),
)

In [ ]:
llm = ChatGoogleGenerativeAI(
    model=gemini2,
    cached_content=cache.name,
)

response = llm.invoke("Compare the revenue growth between Q1 and Q2 2024.")

In [ ]:
# print(response.text)
from IPython.display import Markdown, display
display(Markdown(response.text))

In [ ]:
response.usage_metadata

In [ ]:
response = llm.invoke("Provide a detailed analysis of Apple's Q1 and Q2 2024 earnings with key financial metrics, revenue comparison, and growth trends. Format this as bullet points suitable for an infographic.")

In [ ]:
display(Markdown(response.text))

In [ ]:
response.usage_metadata

### Image Generation

In [ ]:
from langchain_google_genai import Modality
from IPython.display import Image, display

aspect_ratio = "16:9"
resolution = "4K"

In [ ]:
image_model = ChatGoogleGenerativeAI(
    model="gemini-3-pro-image-preview",
    model_kwargs={
        "image_config": {
            "aspect_ratio": aspect_ratio,
            "image_size": resolution,
        }
    }
)

image_response = image_model.invoke(
    f"Create a professional infographic with this data:\n\n{response.text}",
    response_modalities=[Modality.TEXT, Modality.IMAGE],
)

In [ ]:
image_response.content_blocks

In [ ]:
image_response

In [ ]:
def get_image_base64(response):
    # Go through each block in the response
    for block in response.content:
        # Check if this block is a dictionary
        if isinstance(block, dict):
            # Check if it has image data
            if "image_url" in block:
                # Extract the URL
                image_url_data = block["image_url"]
                full_url = image_url_data["url"]
                # The URL looks like: "data:image/png;base64,ACTUALBASE64DATA"
                # We only want the part after the comma
                base64_string = full_url.split(",")[1]
                return base64_string


In [ ]:
image_base64 = get_image_base64(image_response)
display(Image(data=base64.b64decode(image_base64), width=800))

In [ ]:
with open("data/images/apple_earnings_infographic.png", "wb") as f:
    f.write(base64.b64decode(image_base64))

### Structured Output

In [ ]:
# Use weather tool for the sample structred output
# fields -> location:str, date:str, temperature:str, condition:str

from pydantic import BaseModel

class WeatherOutput(BaseModel):
    location: str
    date: str
    temperature: str
    condition: str

model = ChatGoogleGenerativeAI(model=gemini2)
model_with_tools = model.bind_tools([base_tools.get_weather])

structured_model = model_with_tools.with_structured_output(WeatherOutput)

In [ ]:

response = structured_model.invoke("what is the weather in Mumbai today?")

In [ ]:
response